<a id="CELL1"></a>
## CELL 1 
Last code modified: Wednesday, August 7, 2019 9:23:22 AM GMT-04:00 DST

In [26]:
"""CELL 1
builds and returns a database session
local assumes a psql instance in a local docker container
only postgres database is supported for configuration_application at this time
"""
"""
gets env-based configuration secret
returns a session to the configuration db
for dev env it pre-populates the database with helper and seed data
"""
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

2019-08-07 13:27:32,728 - core.helpers.session_helper.SessionHelper - INFO - Creating session for dev environment...
2019-08-07 13:27:32,757 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating administrator mocks.
2019-08-07 13:27:32,762 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating administrator mocks.
2019-08-07 13:27:32,763 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating pharmaceutical company mocks.
2019-08-07 13:27:32,766 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating pharmaceutical company mocks.
2019-08-07 13:27:32,767 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating brand mocks.
2019-08-07 13:27:32,770 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating brand mocks.
2019-08-07 13:27:32,771 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating segment mocks.
2019-08-07 13:27:3

## CONFIGURATION - PLEASE TOUCH
### <font color=pink>This cell will be off in production as configurations will come from the configuration postgres DB</color>

In [27]:
"""
************ CONFIGURATION - PLEASE TOUCH **************
Pipeline Builder configuration: creates configurations from variables specified here!!
This cell will be off in production as configurations will come from the configuration postgres DB.
"""
"""
PIPELINE STATE:

raw-->ingest-->master-->enhance-->enrich-->metrics-->dimensional

"""
# config vars: this dataset
config_pharma = "sun" # the pharmaceutical company which owns {brand}
config_brand = "ilumya" # the brand this pipeline operates on
config_state = "enrich" # the state this transform runs in
config_name = "enrich_patient_journey_hierarchy" # the name of this transform, which is the name of this notebook without .ipynb

# input vars: dataset to fetch. 
# Recall that a contract published to S3 has a key format branch/pharma/brand/state/name
input_branch = "sun-extract-validation"
# None
# if None, input_branch is automagically set to your working branch
input_pharma = "sun"
input_brand = "ilumya"
input_state = "ingest"
input_name = "symphony_health_association_ingest_column_mapping"

#This contract defines the base of the output structure of data into S3.
#
#contract structure in s3: 
#s3:// {ENV} / {BRANCH} / {PARENT} / {CHILD} / {STATE} / {name of input}
#
#ENV - environment Must be one of development, uat, production.
#Prefixed with integrichain- due to global unique reqirement
#BRANCH - the software branch for development this will be the working pull request (eg pr-225)
#in uat this will be edge, in production this will be master
#PARENT - The top level source identifier
#this is generally the customer (and it is aliased as such) but can be IntegriChain for internal sources,
#or another aggregator for future-proofing
#CHILD - The sub level source identifier, generally the brand (and is aliased as such)
#STATE - One of: raw, ingest, master, enhance, enrich, metrics


In [28]:
import logging
logging.getLogger().setLevel(logging.DEBUG)
log = logging.getLogger()

### <font color=orange>SETUP - DON'T TOUCH </font>
Populating config mocker based on config parameters...

In [29]:
"""
************ SETUP - DON'T TOUCH **************
Populating config mocker based on config parameters...
"""
import core.helpers.pipeline_builder as builder

ids = builder.build(config_pharma, config_brand, config_state, config_name, session)
"""
RETURNS: A list of 2 items: [transformation_id, run_id] where transformation_id corresponds
to the configuration created/found for {transformation} and run_id is a randomly generated 6 digit
number (to avoid publishing to the same place with the same dataset)
"""
transform_id = ids[0]
run_id = ids[1]

2019-08-07 13:27:50,691 - core.logging - DEBUG - Adding/getting mocks for specified configurations...
2019-08-07 13:27:50,733 - core.logging - DEBUG - Done. Creating mock run event and committing results to configuration mocker.


In [18]:
# debug only
# e.g.
# 6
# 136126
log.debug(f'Transform Id:{transform_id} Run Id:{run_id}')

2019-08-07 12:09:32,453 - root - DEBUG - Transform Id:6 Run Id:104404


### <font color=orange>SETUP - DON'T TOUCH </font>
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 


In [30]:
"""************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET, BATCH_JOB_QUEUE
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract


db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


In [31]:
#debug only
# e.g:
#DC-578_PatientStatus
#ichain-dev
#dev-core
log.debug(f'Branch name:{BRANCH_NAME} Env Bucket:{ENV_BUCKET} Batch Job Queue:{BATCH_JOB_QUEUE}')

2019-08-07 13:28:54,614 - root - DEBUG - Branch name:DC-546_ENRICH_PATIENT_JOURNEY_HIERARCHY Env Bucket:ichain-dev Batch Job Queue:dev-core


***
# CORE Cartridge Notebook::[master_patient_status]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

## CONFIGURATION - VARIABLES - PLEASE TOUCH

# TRANSFORM

In [71]:
""" 
CONFIGURATION ********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>
e.g.
class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""
"""
imports
"""
import pandas as pd
from core.logging import get_logger
 
class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    ''' 
    col_status: str 
    col_substatus: str 
    customer_name: str
        
    def enrich_status_substatus(self):
        customer_name = transform.customer_name
        try:
            if customer_name=='sun':
                #substatus_dict = Transform.master_substatus_sun()
                #substatus_conversion_dict = Transform.master_substatus_conversion_sun()
                pass
            elif customer_name=='bi':
                #substatus_dict = Transform.master_substatus_bi()
                pass
            elif customer_name=='alkermes':
                #substatus_dict = Transform.master_substatus_alkermes()
                pass
            else:
                logger.exception('expecting customer name as sun bi or alkermes')
                raise Exception('expecting customer name as sun bi or alkermes') 
        except Exception as e:
            go = False 
            logger.exception(f'exception:{e}')
            raise Exception(f'raise exception:{e}')    
        return    
    
    
    def enrich_patient_journey_hierarchy(self,df):
        try:        
            go = False # assume things are not working YET.
           
            dffail = pd.DataFrame() # initialize df for fails
            
            # master data golden status
            #working draft] Gold Domain of patient status
            logger.info('try:')
            status_dict = {}
            #status_dict[1]='ACTIVE'
            #status_dict[2]='CANCELLED'
            #status_dict[3]='DISCONTINUED'
            #status_dict[4]='DENIED'
            #status_dict[5]='PENDING'
            
            # store the golden values in a list
            status_list = list(status_dict.values())           
            
            # log metadata 
            
            logger.info(f'Gold Domain List:{status_list}')
            
            # df in
            dfShape = df.shape
            logger.info(f'df in  shape: {dfShape[0]} {dfShape[1]}')
            logger.info(f'df in {df.head()}') 
            
            # are we expecting certain column names? YES 
            statusColNameExpected = transform.col_status
            substatusColNameExpected = transform.col_substatus
            
            logger.info(f'expecting column name  referral as:{statusColNameExpected}')
            logger.info(f'expecting column name  referral as:{substatusColNameExpected}')
            columnNamesArr = df.columns.values.tolist()
            logger.info(f'df column names:{columnNamesArr}')
            
            if statusColNameExpected in columnNamesArr and substatusColNameExpected in columnNamesArr:
 
                # apply Upper Case to col(s) values of interest
                # apply strip  to col(s) values of interest
                df[statusColNameExpected]= df[statusColNameExpected].apply(lambda x: x.upper() if x is not None else x)   
                df[statusColNameExpected]= df[statusColNameExpected].apply(lambda x: x.strip() if x is not None else x)
                df[substatusColNameExpected]= df[substatusColNameExpected].apply(lambda x: x.upper() if x is not None else x)   
                df[substatusColNameExpected]= df[substatusColNameExpected].apply(lambda x: x.strip() if x is not None else x)
                df[substatusColNameExpected]= df[substatusColNameExpected].apply(lambda x: x.replace('_',' ').replace('\r', '').replace('\t', '').replace('\w', '') if x is not None else x)
    
                # what fails
                dffail =  pd.DataFrame()
                # dffail = df[~df[statusColNameExpected].isin(status_list)]
                # apply master selection for the column of interest
                # what passes
                df = pd.DataFrame()
                #df = df[df[statusColNameExpected].isin(status_list)]
                
                # meta data log for what comes out of the function pass and fail df
                dfOutSize = df.size
                dfOutShape = df.shape
                dffailSize = dffail.size
                dffailShape = dffail.shape
                logger.info(f'df in   shape: {dfShape[0]} {dfShape[1]}')                
                logger.info(f'df pass shape: {dfOutShape[0]} {dfOutShape[1]}')
                logger.info(f'df fail shape: {dffailShape[0]} {dffailShape[1]}')
                logger.info(f'df pass {df.head()}')
                logger.info(f'df fail {dffail.head()}')  
                go = True
            else:
                go = False  
                logger.exception('expecting column names for patient status substatus if/else exception raise')
                raise Exception('expecting column names for patient status substatus if/else exception raise')              
        except Exception as e:
            go = False  
            logger.exception(f'exception:{e}')
            raise Exception(str(e))
        else:
            pass
        finally:
            pass
        return df.copy(),dffail.copy(),go
                

transform = Transform()
logger = get_logger(f"core.transforms.{transform.state}.{transform.name}")

### *Please place your value assignments for development below*
### <font color=pink>This cell will be turned off in production, Engineering will set to pull from the configuration</color>

In [45]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull from the configuration application instead
## For the last example, this could look like...
## transform.some_ratio = 0.6
## transform.site_name = "WALGREENS"

transform.customer_name = 'sun'
transform.col_status = 'status_code'
transform.col_substatus = 'sub_status'



### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

## Planned

We map status/sub-status combinations to a patient journey model in order to provide actionable insights to our customers.

Definition of Done:

- by status/sub-status combination - map to a patient journey bucket defined in a bridge table.
- process to kick out any statuses/sub-statuses that are NOT mapped. 

**Transform assumes MASTER patient status and MASTER substatus transforms at a minimum have been run.**



### FETCH DATA - TOUCH, BUT CAREFULLY
### <font color=pink>This cell will be turned off in production, as the input_contract will be handled by the pipeline</color>

In [35]:
"""
************ FETCH DATA - TOUCH, BUT CAREFULLY **************
This cell will be turned off in production, as the input_contract will be handled by the pipeline.
"""
logger.info("FETCH DATA CELL - TOUCH - This cell will be turned off in production, as the input_contract will be handled by the pipeline. ")

# for testing / development only
run_id = 3

if not input_branch:
    input_branch = BRANCH_NAME
input_contract = DatasetContract(branch=input_branch,
                                 state=input_state, 
                                 parent=input_pharma, 
                                 child=input_brand, 
                                 dataset=input_name)
run_filter = []
run_filter.append(dict(partition="__metadata_run_id", comparison="==", values=[run_id]))
# IF YOU HAVE PUBLISHED DATA MULTIPLE TIMES, uncomment the above line and change the int to the run_id to fetch.
# Otherwise, you will have duplicate values in your fetched dataset!

# bypass/comment out when unit testing individual parquet files
df = input_contract.fetch(filters=run_filter)



2019-08-07 13:34:25,584 - core.transforms.enrich.enrich_patient_journey_hierarchy - INFO - FETCH DATA CELL - TOUCH - This cell will be turned off in production, as the input_contract will be handled by the pipeline. 
2019-08-07 13:34:25,588 - core.dataset_contract.DatasetContract - INFO - Fetching dataframe from s3 location s3://ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping.
2019-08-07 13:34:25,743 - urllib3.util.retry - DEBUG - Converted retries value: False -> Retry(total=False, connect=None, read=None, redirect=0, status=None)
2019-08-07 13:34:25,744 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): ichain-dev.s3.amazonaws.com:443
2019-08-07 13:34:25,857 - urllib3.connectionpool - DEBUG - https://ichain-dev.s3.amazonaws.com:443 "GET /?prefix=sun-extract-validation%2Fsun%2Filumya%2Fingest%2Fsymphony_health_association_ingest_column_mapping&encoding-type=url HTTP/1.1" 200 None
2019-08-07 13:34:25,878 - urllib3

## *<font color=grey>unit test development only*</font>
*<font color=grey>The next **5** cells will be deleted in production.* </font>

In [ ]:
#import pyarrow.parquet as pq
#import s3fs

#def pandas_from_parquet_s3(file_path):  
#    s3 = s3fs.S3FileSystem()
#    df = (
#        pq
#        .ParquetDataset(file_path, filesystem=s3)
#        .read_pandas()
#        .to_pandas()
#    )    
#    return df

In [ ]:
# unit test/development 
# isolate on individual parquet files
#TEST 1
#df = pandas_from_parquet_s3('ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping/__metadata_run_id=3/90ca3aa7b0bb4246a281591b013ff54e.parquet')
# TEST 2
#df = pandas_from_parquet_s3('ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping/__metadata_run_id=3/d7ad974cef284e19aa7b5ac410220b96.parquet')
# TEST 3
#df = pandas_from_parquet_s3('ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping/__metadata_run_id=3/1a6ffd3598d442e38fbba66ea85a55a2.parquet')
# TEST 4
#df = pandas_from_parquet_s3('ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping/__metadata_run_id=3/5c00059d9fc04b0e8bc4ce764c50f3fb.parquet')
# TEST 5
# df = pandas_from_parquet_s3('ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping/__metadata_run_id=3/6eceb7ce59bd4dec8720316b4209b0e3.parquet')
# THEN ALL TEST use 
# then use the FETCH DATA - TOUCH, BUT CAREFULLY CELL

In [37]:
# unit test/development only
# before shot unit testing only
dfSize = df.size
dfShape = df.shape
print('shape: {} {}'.format(dfShape[0],dfShape[1])) 

shape: 24457 72


In [ ]:
# unit test/development only
# needed to see the col(s) of interest
#pd.set_option('display.max_columns', 50)

In [36]:
## unit test/development only
df.head()

,rec_date,pharm_code,pharm_npi,transtype,pharm_transaction_id,trans_seq,ref_source,ref_date,program_id,pharmacy_id,...,copay_assist_amount,oth_payer_amt,xfer_pharmname,msa_patient_id,msa_patient_bmap,__metadata_run_timestamp,__metadata_app_version,__metadata_output_contract,__metadata_transform_timestamp,__metadata_run_id
0,20181024115959,ACCREDO,1346208949,COM,279133432018102401,0,DIRECT,20181019120000,None,27913343,...,None,None,None,None,NNNNV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3
1,20181025115959,ACCREDO,1346208949,COM,278370982018102502,0,DIRECT,20181022120000,None,27837098,...,None,None,None,None,NNNVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3
2,20181029115959,ACCREDO,1346208949,COM,279181482018102903,0,DIRECT,20181024120000,None,27918148,...,None,None,None,None,NNNVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3
3,20181102115959,ACCREDO,1346208949,COM,267244982018110204,0,DIRECT,20181030120000,None,26724498,...,None,None,None,None,NNNVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3
4,20181106115959,ACCREDO,1346208949,COM,160618142018110605,0,DIRECT,20181102120000,None,16061814,...,None,None,None,None,NNNVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3


# <font color=red>**CALL**</font> THE TRANSFORM

In [46]:
### Use the variables above to execute your transformation.
### the final output needs to be a variable named final_dataframe
logger.info("CALL THE TRANSFORM - execute your transformation - the final output needs to be a variable named final_dataframe")

final_dataframe, final_fail, go = transform.enrich_patient_journey_hierarchy(df)

if go==True:
    logger.info("CALL THE TRANSFORM -  go no go = GO")
elif go==False:
    logger.info("CALL THE TRANSFORM -  go no go = NO go")
else:
    go=False
    logger.info("CALL THE TRANSFORM -  go no go = unknown make it NO go")
        

2019-08-07 13:41:21,233 - core.transforms.enrich.enrich_patient_journey_hierarchy - INFO - CALL THE TRANSFORM - execute your transformation - the final output needs to be a variable named final_dataframe
2019-08-07 13:41:21,238 - core.transforms.enrich.enrich_patient_journey_hierarchy - INFO - try:
2019-08-07 13:41:21,242 - core.transforms.enrich.enrich_patient_journey_hierarchy - INFO - Gold Domain List:[]
2019-08-07 13:41:21,243 - core.transforms.enrich.enrich_patient_journey_hierarchy - INFO - df in  shape: 24457 72
2019-08-07 13:41:21,263 - core.transforms.enrich.enrich_patient_journey_hierarchy - INFO - df in          rec_date pharm_code   pharm_npi transtype pharm_transaction_id  \
0  20181024115959    ACCREDO  1346208949       COM   279133432018102401   
1  20181025115959    ACCREDO  1346208949       COM   278370982018102502   
2  20181029115959    ACCREDO  1346208949       COM   279181482018102903   
3  20181102115959    ACCREDO  1346208949       COM   267244982018110204   
4  

### *<font color=grey>unittest python*</font>

In [62]:
import unittest

def ut_shapeInVsOut(df,final_dataframe):
    """
    assertion will change based on coding state and input datasets
    inital template (gutted) expect ... FAIL
    """
    return df.shape == final_dataframe.shape

def ut_shape_passVsfail(df_pass,df_fail):
    """
    assertion will change based on coding state and input datasets
    inital template (gutted) expect ... ok
    """
    return df_pass.shape == df_fail.shape

def ut_shape_InVsfail(df,df_fail):
    """
    assertion will change based on coding state and input datasets
    inital template (gutted) expect ... FAIL
    """
    return df.shape == df_fail.shape

class TestNotebook(unittest.TestCase):
    
    def test_ut_shapeInVsOut(self):
        
        self.assertEqual(ut_shapeInputVsOut(df,final_dataframe),True)
        
    def test_ut_shape_InVsfail(self):
        
        self.assertEqual(ut_shape_InVsfail(df,final_fail),True)     
        
    def test_ut_shape_passVsfail(self):
        
        self.assertEqual(ut_shape_passVsfail(final_dataframe,final_fail),True)    
        
     
                
"""
"""
# for development only
unittest.main(argv=[''], verbosity= 2, exit=False)        
    

test_ut_shapeInVsOut (__main__.TestNotebook) ... FAIL
test_ut_shape_InVsfail (__main__.TestNotebook) ... FAIL
test_ut_shape_passVsfail (__main__.TestNotebook) ... ok

FAIL: test_ut_shapeInVsOut (__main__.TestNotebook)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-62-92bb1d26151d>", line 28, in test_ut_shapeInVsOut
    self.assertEqual(ut_shapeInputVsOut(df,final_dataframe),True)
AssertionError: False != True

FAIL: test_ut_shape_InVsfail (__main__.TestNotebook)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-62-92bb1d26151d>", line 32, in test_ut_shape_InVsfail
    self.assertEqual(ut_shape_InVsfail(df,final_fail),True)
AssertionError: False != True

----------------------------------------------------------------------
Ran 3 tests in 0.009s

FAILED (failures=2)


In [ ]:
# untit test/development only look at the fails
#final_fail.head()
#final_fail[transform.col_status]

In [ ]:
# untit test/development only look at the pass(es)
#final_dataframe.head()
#final_dataframe[transform.col_status]

# **publish**
### Writing to S3
Invoke the `publish()` command to write to a given contract. Some things to know:
- To invoke publish a contract must be at the grain of dataset. This is because file names will be set by the dataframe=\>parquet conversion. 
- publish only accepts a pandas dataframe.
- publish does not allow for timedelta data types at this time (this is missing functionality in pyarrow).
- publish handles partitioning the data as per contract, creating file paths, and creating the binary parquet files in S3, as well as the needed metadata. <br>
**- by default, all datasets include a single partition, \_\_metadata\_run\_id, the RunEvent ID of an executed pipeline**

In [ ]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
if go==True:
    logger.info("PUBLISH - that's it - its a GO - just provide the final dataframe to the var final_dataframe and we take it from there")
    transform.publish_contract.publish(final_dataframe, run_id, session)
elif go==False:
    logger.info("PUBLISH -  go no go = NO go -  so DONT publish")
else:
    go=False
    logger.info("PUBLISH -  go no go = unknown make it NO go - so DONT publish")    
session.close()

***